# Messages in Autogen v0.4
We can image messages as the way agent communication - text our friend

In [25]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage,MultiModalMessage
from autogen_core import Image as AGImage

from PIL import Image
from io import BytesIO

import requests
from dotenv import load_dotenv
import os

In [26]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("OPENAI_API_KEY environment variables is not set.")
model_client = OpenAIChatCompletionClient(model='gpt-4o',api_key=api_key)


# Simplest Type of Message - Text Message

In [27]:
agent = AssistantAgent(
    name = "text_agent",
    model_client=model_client,
    system_message='You are a helpful assistant. answer question accurately'
)

In [28]:
async def test_text_messages():
    text_msg = TextMessage(content='What is the capital of USA?', source='user')
    result = await agent.run(task=text_msg)
    print(result.messages[-1].content)

await test_text_messages()
    

The capital of the United States is Washington, D.C.


# Multi Modal Messages

In [29]:
async def test_multi_modal():
    # Retrieve image
    #response = requests.get('https://picsum.photos/id/237/200/300') # black labrador on floor
    response = requests.get('https://picsum.photos/id/15/200/300') # waterfall near rocks
    # Convert image to bytes
    pil_image = Image.open(BytesIO(response.content))
    ag_image = AGImage(pil_image)

    # Create multi modal image
    multi_modal_msg = MultiModalMessage(
        content = ['What is in the image?', ag_image],
        source='user'
    )

    result = await agent.run(task=multi_modal_msg)
    print(result.messages[-1].content)
await test_multi_modal()

The image shows a waterfall with water cascading down a rocky cliff, surrounded by lush green vegetation. In the foreground, there are rocks and a stream.


# Structured Output
"Labrador is a breed of Dog"
{'content':'Dog','species':'Labra'}

In [30]:
# NOTE: Use pydatnic model
from pydantic import BaseModel

In [31]:
# Basically, inherit from Pydantic base model, and specify what outputs want from model response
# So want a name (string), color (string), and miles (integer)
class PlanetInfo(BaseModel):
    name: str
    color: str
    distance_miles:int

In [32]:
model_client = OpenAIChatCompletionClient(
    model = 'gpt-4o',
    api_key=api_key,
    response_format=PlanetInfo
)

In [33]:
unstructured_model_client = OpenAIChatCompletionClient(
    model = 'gpt-4o',
    api_key=api_key
)

In [34]:
# Inject model (brain) into agent (digital human)
agent = AssistantAgent(
    name='planet_agent',
    model_client=unstructured_model_client,
    system_message="You are a helpful assistant that provides information about planets. in the structure JSON" \
    "{ name :str" \
    "age : int" \
    "}"
)

In [35]:
# Make call to agent (unstructured model client)
async def test_unstructured_output():
    task = TextMessage(content = "Please provide information about Mars.", source = 'User')
    result = await agent.run(task=task)
    structured_response = result.messages[-1].content
    print(structured_response)

await test_unstructured_output()

```json
{
  "name": "Mars",
  "average_distance_from_sun_km": 227943824
}
```


In [44]:
# Make call to agent (with sturctured model client)
agent = AssistantAgent(
    name='planet_agent',
    model_client=model_client,
    system_message="You are a helpful assistant that provides information about planets. in the structure JSON" \
    "{ name :str" \
    "age : int" \
    "}"
)

async def test_structured_output():
    task = TextMessage(content = "Please provide information about Mars.", source = 'User')
    result = await agent.run(task=task)
    response = result.messages[-1].content
    print(response)

    return response

resultJson = await test_structured_output()

{"name":"Mars","color":"Red","distance_miles":141600000}


In [ ]:
print(resultJson)


{"name":"Mars","color":"Red","distance_miles":141600000}
